# Import

In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import pandas as pd
import numpy as np
import time
import jsonlines

In [45]:
%load_ext autoreload
%autoreload 2

from seewhence import models
from seewhence import train

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
# Directories & Paths
OUTPUT_DIR = 'data'

# Load and split Sarcasm Data

In [47]:
# reddit_path = '/home/ben/data/acl_2020/reddit.jsonl'
# twitter_path = '/home/ben/data/acl_2020/twitter.jsonl'
# train_split = 0.8
# %run ./preprocessing.ipynb

# Torch Text Dataset

In [48]:
# Fields
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)
fields = {'response': ('text', TEXT), 'label': ('label', LABEL)}

In [49]:
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = OUTPUT_DIR,
                                        train = 'train.csv',
                                        validation ='valid.csv',
                                        test = 'test.csv',
                                        format = 'csv',
                                        fields = fields
)

## Build Vocab

In [50]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.twitter.27B.200d", 
                 unk_init = torch.Tensor.normal_)

In [51]:
LABEL.build_vocab(train_data)

# Iterator

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

# LSTM

In [53]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
output_path = 'models_lstm_'


model = models.RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)



In [54]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [55]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [59]:
N_EPOCHS = 7

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train.train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = train.evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = train.epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        if output_path:
            torch.save(model.state_dict(), output_path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 7s
	Train Loss: 0.662 | Train Acc: 59.43%
	 Val. Loss: 0.596 |  Val. Acc: 66.83%
Epoch: 02 | Epoch Time: 0m 5s
	Train Loss: 0.596 | Train Acc: 67.55%
	 Val. Loss: 0.590 |  Val. Acc: 69.80%
Epoch: 03 | Epoch Time: 0m 5s
	Train Loss: 0.551 | Train Acc: 71.76%
	 Val. Loss: 0.616 |  Val. Acc: 69.50%
Epoch: 04 | Epoch Time: 0m 5s
	Train Loss: 0.513 | Train Acc: 74.46%
	 Val. Loss: 0.558 |  Val. Acc: 70.47%
Epoch: 05 | Epoch Time: 0m 5s
	Train Loss: 0.437 | Train Acc: 79.85%
	 Val. Loss: 0.586 |  Val. Acc: 72.04%
Epoch: 06 | Epoch Time: 0m 5s
	Train Loss: 0.379 | Train Acc: 83.12%
	 Val. Loss: 0.660 |  Val. Acc: 70.90%
Epoch: 07 | Epoch Time: 0m 5s
	Train Loss: 0.337 | Train Acc: 84.83%
	 Val. Loss: 0.658 |  Val. Acc: 71.66%


In [61]:
model.load_state_dict(torch.load(output_path))
test_loss, test_acc = train.evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.573 | Test Acc: 69.95%
